# پروژه سوم - Naive Bayes Classifier 
## نازنین یوسفیان
### 810197610

## هدف
### در این پروژه قصد داریم که تعدادی کامنت را تحلیل کرده و مشخص کنیم که با توجه به متن آن، کامنت مثبتی بوده است یا خیر. در این جا از الگوریتم Naive Bayes Classifier استفاده می کنیم.


## مقدمه
### دو فایل به نام های *comment_train* و *comment_test* در اختیار ما قرار گرفته اند. ابتدا باید کامنت های موجود در train را تحلیل کرده و جدول احتمالات هر کلمه به شرط recommended بود و نبودن را به دست آوریم. سپس برای هر کامنت در test تصمیم بگیریم که چه مقداری برای آن مناسب است. 

### توضیح پروژه



In [59]:
from __future__ import unicode_literals
from hazm import *
import pandas as pd
import time
import scipy.stats as ss
import matplotlib.pyplot as plt
from collections import defaultdict
import math
import collections
file_name1 = 'comment_train.csv'
file_name2 = 'comment_test.csv'
data = pd.read_csv(file_name1)
test = pd.read_csv(file_name2)
data['new'] = data['comment'] + ' ' + data['title']
test['new'] = test['comment'] + ' ' + test['title']



### پیش پردازش داده
برای تحلیل درست تر، بهتر است که از پیش پردازش روی داده ها استفاده کنیم. در مرحله اول ستون کامنت و تیتر را با یکدیگر ترکیب کرده و به عنوان یک ستون در نظر می گیریم. ابتدا از تابع normalize استفاده می کنیم که نیم فاصله ها و فاصله ها را در متن به درستی مشخص می کند تا در مراحل بعد از توابع دیگر نتیجه بهتری بگیریم.

In [60]:
def normal(x):
    normalizer = Normalizer()
    return normalizer.normalize(x)
    
data['new'] = data['new'].apply(normal)
test['new'] = test['new'].apply(normal)

   
    


در مرحله بعدی متونی که داریم را به کلمات تشکیل دهنده آن با استفاده از تابع *()word_tokenize* تجزیه می کنیم.

In [61]:
def token(x):
    return word_tokenize(x)
data['new'] = data['new'].apply(token)
test['new'] = test['new'].apply(token)


حال تابع *()lemmatize* را روی کلمات مشخص شده صدا می زنیم. این تابع کمک می کند که اطلاعات اضافه کلمات مانند پیشوندها ، نشانه های جمع و ...حذف شوند. هم چنین برای فعل ها بن ماضی و مضارع آن را به ما بر می گرداند که برای تحلیل مفیدتر هستند. زیرا فرقی نمی کند که از یک فعل در چه زمانی استفاده شده باشد و یا چه شناسه ای داشته باشد. هم چنین در واژه ها نشانه های جمع برای ما تفاوتی ایجاد نمی کنند و بهتر است که یک کلمه و جمع آن از یکدیگر متمایز نباشند. 
در این جا چون نظرات به زبان محاوره ای نوشته شده اند، در بعضی از موارد نمی تواند کلمات را به درستی به ما برگرداند ولی در موارد دیگر به درستی کار می کند و دقت آن برای ما مطلوب است. 

In [62]:
lemmatizer = Lemmatizer()

def lemma(x):
    for i in x:
        lemmatizer.lemmatize(i)
    return x

data['new'] = data['new'].apply(lemma)
test['new'] = test['new'].apply(lemma)



پیش پردازش دیگری که از آن می توانیم استفاده کنیم حذف کلمات پرتکرار است. به طور مثال حروف اضافه در متن ها زیاد به کار می روند ولی تاثیری در تحلیلی که انجام می دهیم ندارند و بر اساس آن ها نمی توان تصمیم گرفت که آن نظر مثبت محسوب می شود یا خیر. در  *stopwords_list* لیستی از این کلمات وجود دارد و ما آن ها را از نظرات پاک کرده ایم. کلماتی مانند *عالی* و *بسیار* نیز در این لیست وجود دارند که حذف نکردن آن ها می تواند برای تحلیل بهتر باشد ولی با حذف آن ها نیز تحلیل ما از دقت قابل قبولی برخوردار است. 

In [63]:
stopwords = set(stopwords_list())
def del_stopword(x):
    new_set = x.copy()
    for i in x:
        if i in stopwords:
             new_set.remove(i)
    return new_set
            
data['new'] = data['new'].apply(del_stopword)
test['new'] = test['new'].apply(del_stopword)


### فرآیند حل مسئله
برای حل این مسئله از مدل bag of words استفاده می کنیم به این صورت که هر کلمه را مستقل از جایگاهش در نظر می گیریم. هر کلمه و اینکه آن کامنت پیشنهاد شده است یا نه یک feature محسوب می شود. در Naive Bayes پیشنهاد شدن یا نشدن را پدر همه feature های دیگر در نظر می گیریم و این فرض را داریم که فرزندانش به شرط دانستن پدر از یکدیگر مستقل هستند که فرض قوی ای است و در واقعیت این گونه نیست. در یک متن کلمات به یکدیگر وابسته اند ولی با این فرض هم نتیجه ای که به دست می آید از دقت بالایی برخوردار است. پس اگر اینکه کامنتی مثبت باشد را c در نظر بگیریم، می خواهیم P(c|X) را حساب کنیم که X کلماتی هستند که در آن کامنت وجود دارند.
پس در کامنت جدیدی که به دست ما می رسد، می خواهیم بدانیم با توجه به کلماتی که در آن به کار رفته است کامنت مثبتی است یا خیر. برای به دست آوردن این احتمال ابتدا باید احتمال p(c,X) را حساب کنیم که چون x ها به شرط دانستن c از یکدیگر مستقل اند این احتمال به صورت زیر ساده می شود:

p(c | X) = p(c) * p(x1 | c) * p(x2 | c) *…

در اینجا posterior probability احتمال مثبت یا منفی بودن کامنت در فایل test به شرط کلماتی است که در آن به کار رفته است. Likelihood احتمال وجودهر کلمه در کامنت به شرط مثبت یا منفی بودن کامنت است که از تحلیل داده های فایل train به دست می آوریم. Class Prior Probability احتمال مثبت بودن یا منفی بودن یک کامنت است که برای مثبت ها تعداد کل recommended ها را در فایل train به کل کامنت ها تقسیم می کنیم ت احتمال آن بدست بیاید. به همین صورت احتمال منفی بودن به دست می آید. Predictor Prior Probability احتمال وجود یک کلمه در کل کامنت هاست.  

با استفاده از  فایل *comment_train* باید احتمالاتی که در سمت راست تساوی آمده اند را محاسبه کنیم. ابتدا باید در سطرهایی که recommended هستند کلمات متمایز به کار رفته را جدا کنیم و به ازای هر کدام حساب کنیم که چند بار تکرار شده اند. هم چنین باید این محاسبات را برای سطرهای not_recommended تکرار کنیم. این گونه likelihood را محاسبه می کنیم. سپس باید ببینیم که به ازای همه کامنت ها چند درصدشان مثبت و چند درصد منفی بوده اند و بر اساس آن احتمال class prior probability را حساب کنیم.  

In [64]:
recom_words = []
notrecom_words = []
recom = len(data[data['recommend'] == 'recommended'])
not_recom = len(data[data['recommend'] == 'not_recommended'])
total = recom + not_recom
def add_recom_words(x):
    for item in x:
        recom_words.append(item)
def add_notrecom_words(x):
    for item in x:
        notrecom_words.append(item)


data['new'][data['recommend'] == 'recommended'].apply(add_recom_words)
data['new'][data['recommend'] == 'not_recommended'].apply(add_notrecom_words)
recom_table = collections.Counter(recom_words)
notrecom_table = collections.Counter(notrecom_words)

###  Additive Smoothing 
اگر در داده های train کلمه ای در دسته recommended باشد و در دسته not_recommended نباشد، احتمال p(x | recommended) محاسبه نشده و صفر در نظر گرفته می شود. حال اگر به کامنتی در فایل test برخورد کنیم که این کلمه در آن استفاده شده است چون احتمال p(x | recommended) صفر در نظر گرفته شده است باعث می شود احتمال recommended بود آن کامنت صفر شود و not_recommended در نظر گرفته شود. اینگونه به بقیه کلماتی که در آن کامنت وجود داشته اند توجهی نمی شود و صرفا بر اساس یک کلمه تصمیم گیری می شود. همین اتفاق برای وقتی که کلمه در هیچ کدام از دسته ها نباشد نیز می افتد. 


برای حل این مشکل از روش Additive Smoothing استفاده می شود. در این روش یک hyper parameter به نام α در نظر گرفته می شود. عدد α به صورت احتمالات و عدد α * k به مخرج آن ها اضافه می شوند و احتمالات به صورت زیر محاسبه می شوند:

p(x | c) = ( p(x,c) + α ) / ( p(c) + k * α )

که در اینجا k تعداد کلمه هایی است که در دسته c قرار دارند. دلیل اضافه کردن k * α به جای α در مخرج این است که چون این کلمه، کلمه جدیدی است و هیج پیش زمینه ای راجع به آن نداریم پس احتمال آن را به طور یکنواخت روی k دسته بندی حساب می کنیم. با این روش دیگر به احتمال 0 در محاسبات برخورد نمی کنیم  و باعث می شود که بتوانیم بر اساس کلمات دیگری که در کامنت وجود دارند و احتمالشان را می دانیم تصمیم گیری کنیم.

In [65]:
def predict(x):
    recom_prob = math.log(recom / total)
    notrecom_prob = math.log(not_recom / total)
    alpha = 0.05
    for word in x:
        if word in recom_table:
            recom_prob += math.log(recom_table[word] / recom)
        else:
            recom_prob += math.log(alpha / (recom + alpha * total))
            
            
        if word in notrecom_table:
            notrecom_prob += math.log(notrecom_table[word] / not_recom)
        else:
            notrecom_prob += math.log(alpha / (not_recom + alpha * total))
           
    if recom_prob > notrecom_prob:
        return 'recommended'
    else:
         return 'not_recommended'
    
            
    
    
test['predict'] = test['new'].apply(predict)
accuracy = len(test[test['recommend'] == test['predict']]) / len(test)
print('Accuracy: ', accuracy)
precision = len(test[(test['recommend'] == test['predict']) & (test['predict'] == 'recommended')]) / len(test[test['predict'] == 'recommended'])
print('Precision: ', precision)
recall =  len(test[(test['recommend'] == test['predict']) & (test['predict'] == 'recommended')]) / len(test[test['recommend'] == 'recommended'])
print('Recall: ', recall)
f1 = 2 * (precision * recall) / (precision + recall)
print('F1', f1)



Accuracy:  0.93625
Precision:  0.9204819277108434
Recall:  0.955
F1 0.9374233128834355


precision نسبت تعداد کامنت هایی که درست recommended تشخیص داده ایم را به کل کامنت هایی که recommended تشخیص داده ایم بیان می کند در حالی که recall نسبت تعداد کامنت هایی که درست recommended تشخیص داده ایم را به کل کامنت هایی که در واقعیت recommended بوده اند بیان می کند. در حالت هایی که اشتباه مثبت تشخیص دادن ( یعنی حالتی که در واقعیت منفی بوده و ما به اشتباه مثبت تشخیص می دهیم) هزینه زیادی برای ما دارد مثلا در تشخیص spam بودن یک ایمیل، precision برای ما مهم است و باید درصد بالایی داشته باشد. در حالت هایی که اشتباه منفی تشخیص دادن برای ما هزینه زیادی دارد recall اهمیت بالایی دارد. به همین دلیل باید هر دوی این ویژگی ها را حساب کنیم و بالا بودن درصد یکی لزوما بالا بودن درصد دیگری را به همراه ندارد و بین این دو یک trade-off برقرار است. 

حالتی را در نظر بگیریم که تعداد کمی کامنت را recommended تشخیص داده ایم(مثلا 100 تا) و از این بین تعداد زیادی را به درستی تشخیص داده ایم (مثلا 90 تا). با اینکه معیار precision در این حالت خیلی بالا است ولی recall خیلی کم است و مدل ما خوب کار نمی کند. 

حالت دیگری که ممکن است پیش بیاید و مورد قبول ما نباشد این است که از بین کامنت هایی که در واقیت recommended هستند، تعداد زیادی شان را به درستی تشخیص بدهیم ولی تعداد زیادی از not_recommended ها را نیز recommended تشخیص بدهیم که در اینجا معیار recall مقدار بالایی دارد در صورتی که precision دقت کافی را ندارد.

برای حل مشکلاتی که در بالا ذکر شد، به طور کلی می توانیم از معیار F1 استفاده کنیم که میانگین هارمونیک دو معیار precision و recall است. به جای اینکه به دنبال حالتی باشیم که هم precision و هم recall درصد خوبی داشته باشند، می توانیم حالتی را انتخاب کنیم که F1 بالایی داشته باشد. در واقع F1 یک تعادل بین این دو معیار برقرار می کند.

در مواردی که درست تشخیص دادن مثبت و منفی بودن برای ما مهم تر است، از معیار accuracy استفاده می کنیم و در حالاتی که می خواهیم تشخیص اشتباهمان کمتر باشد، به معیار F1 بیشتر توجه می کنیم. هم چنین اگر توزیع ویژگی یکسان نیست مثلا تعداد recommended ها از not_recommended ها خیلی بیشتر است، F1 ما را به مدل بهتری می رساند.

#### جدول معیار ها در حالات متفاوت 

|  | Accuracy | Precision | Recall | F1|
|--|----------|-----------|--------|---|
|a|0.93|0.90|0.96|0.93|
|b|0.92|0.88|0.98|0.93|
|c|0.91|0.90|0.92|0.91|
|d|0.90|0.87|0.94|0.91|





تاثیر Additive Smoothing در تحلیل درست داده بیشتر از پیش پردازش است. زیرا ممکن است که ما داده ها را به خوبی پیش پردازش کرده و کلمات را به خوبی جایگزین کنیم ولی هنگامی که با کلمه جدیدی برخورد می کنیم که در لیست احتمالاتمان نیست، تمام پردازش هایی که روی بقیه کلمات انجام داده ایم بی تاثیر شده و احتمال نهایی صفر می شود که درصد خطا را بالا می برد. انجام پیش پردازش خطای ما را کمتر می کند و روش هایی که استفاده می کنیم در نتیجه نهایی موثر است. باید دقت شود که در تحلیل داده ها، نوع داده ها از چه دسته ای هستند و بر اساس آن کلمات پرتکرار را انتخاب کرده و حذف کنیم. ممکن است کلمه پرتکراری داشته باشیم که در تحلیل نوعی از داده موثر است و نباید حذف شود و حذف نکردن آن دقت را بالا ببرد در صورتی که در نوع دیگری از داده حذف نکردن این کلمه باعث خطای بیشتری شود.


هم چنین مشاهده می شود که مقدار Recall نسبت به Precision بالاتر است و دقت بیشتری دارد. درصد بالایی از کامنت هایی که recommended بوده اند را به درستی تشخیص داده ایم ولی کامنت هایی که recommended تشخیص داده ایم، بعضی هایشان not_recommended بوده اند و درصد خطایمان در این حالت بیشتر بوده است. مقادیر F1 و Accuracy تقریبا با یکدیگر برابرند و این به این علت است که توزیع داده تقریبا یکسان بوده است.

#### کامنت هایی که اشتباه تشخیص داده شده اند:
1. بعد از ۱ سال استفاده  بعد از ۱ سال استفاده کاملا چسب هاش باز شده و شکلی زشت پیدا کرده مجبور شدم درش بیارم، برش هاشم دقیق نیست خیلی بده تنها خوبیش این بود راحت در آوردم و انداختم تو سطل آشغال
2. عالی است عالی عالی است
3. دیر شارژ میکند اصلا کیفیت ندارد. 20 ساعت طول میکشه تا گوشی رو شارژ کنه
4. روکش بالشتاش اصلا شبیه عکسش نبود صورتی ساده بود روتختیش نسبت به بقیه روتختیا کوچیکتره جوری که از تخت آویز نمیشه لب به لب تخت اندازست ایکاش اون عکسی که میزارن با اونی که میفرستن یکی بود
5. خیالم راحت شد فندک قبلیم مدام فیوز میسوزوند و یک بار شارژر موبایل هم سوزوند ولی با این هیچ مشکلی بوجود نیومده تا الان. کیفیتش خیلی خوبه و لامپ هم داره 

کامنت های 1، 3و 4 به درستی لیبل نخورده اند. یعنی از متن کامنت اینگونه برداشت می شود که باید not_recommended باشند در صورتی که در فایل test لیبل recommended خورده اند و درواقع ما به درستی آن ها را تشخیص داده ایم. 
در مورد دوم *عالی* و *است* جزو کلمات پرتکراری هستند که حذف شده اند و به همین علت نتوانسته ایم به درستی تشخیصشان بدهیم. 
مورد پنجم کلماتی دارد که احتمال اینکه به آن ها در train برخورده باشیم کم است و در واقع پیش زمینه ای برای آن ها نداریم. هم چنین از جمله دوم و سوم به تنهایی ممکن است این برداشت شود که از این محصول رضایت ندارد. هم چنین کلمه *مشکل* احتمال زیادی دارد که در دسته not_recommended باشد.

از مشکلاتی که تحلیل ما دارد این است که هر کلمه را مستقل در نظر می گیریم در صورتی که می دانیم کلمات در یک جمله به یکدیگر وابسته اند و باید به مفهوم جمله توجه شود. هم چنین علائم نگارشی در تحلیل تاثیر زیادی دارند. ممکن است در نظری که recommended است ابتدا ایرادات جزئی محصول گفته شود ولی درکل رضایت از آن اعلام شود که این تحلیل ما را با مشکل رو به رو می کند.


## نتیجه گیری
### مدل Naive Bayes مدل ساده ای است که بر اساس آن می توان تعدادی داده را دسته بندی کرد. الگوریتم آن در عین سادگی از دقت خوبی برخوردار است. از مشکلات این الگوریتم این است که هر کلمه به شرط دانستن ویژگی که به دنبال یافتن آن هستیم، از بقیه کلمات مستقل است و این ممکن است دقت تحلیل ما را پایین بیاورد. در صورتی که داده هایی که برای train انتخاب می شوند، رندوم باشند و توزیع یکسان داشته باشند این الگوریتم می تواند به خوبی عمل کند. 

## منابع

                                                                                                                                               
1. *https://medium.com/analytics-vidhya/intuition-behind-naive-bayes-algorithm-laplace-additive-smoothing-e2cb43a82901*
2. *https://medium.com/syncedreview/applying-multinomial-naive-bayes-to-nlp-problems-a-practical-explanation-4f5271768ebf*
3. *https://www.analyticsvidhya.com/blog/2020/09/precision-recall-machine-learning/*
4. *https://medium.com/@shrutisaxena0617/precision-vs-recall-386cf9f89488*